In [25]:
import aif360
import pandas as pd
from aif360.datasets import BinaryLabelDataset, StandardDataset
from aif360.metrics import ClassificationMetric
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing

In [26]:
df = pd.read_csv('data/dataframe_final.csv')
protected_attributes = ['GENDER', 'LANGUAGE', 'INSURANCE', 'RELIGION', 'ETHNICITY', 'AGE']
df

,SUBJECT_ID,Y_PRED,Y_TRUE,GENDER,LANGUAGE,INSURANCE,RELIGION,ETHNICITY,AGE
0,10011,1,1,0,1,1,1,0,1
1,10026,0,0,0,1,1,0,1,0
2,10030,0,0,1,1,1,1,1,0
3,10042,0,0,1,1,1,1,1,0
4,10094,0,0,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...
3231,9970,0,0,1,1,1,1,1,1
3232,9977,0,0,1,1,1,1,1,0
3233,99863,0,0,1,0,1,1,1,1
3234,99883,0,0,1,1,1,1,1,0


In [91]:
def createBinaryDataset(df, ytrue_df, protected_attribute):
    ytrue_df.rename(columns={0: 'Y_TRUE'}, inplace=True)
    attribute_df = df[protected_attribute].to_frame()
    attribute_df.rename(columns={0: protected_attribute}, inplace=True)
    attribute_df = pd.concat([attribute_df, ytrue_df], axis=1)

    dataset = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=attribute_df,
        label_names=['Y_TRUE'],
        protected_attribute_names=[protected_attribute]
    )

    return dataset

def runMetrics(dataset, dataset_pred, unprivileged_groups, privileged_groups):
    classified_metric = ClassificationMetric(dataset, dataset_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

    print("Accuracy: ", classified_metric.accuracy())
    print("Error Rate Difference: ", classified_metric.difference(classified_metric.error_rate))
    print("False Discovery Rate Difference: ", classified_metric.difference(classified_metric.false_discovery_rate))
    print("False Omission Rate Difference: ", classified_metric.difference(classified_metric.false_omission_rate))
    print("Differential Fairness Bias Amplification: ", classified_metric.differential_fairness_bias_amplification())
    print("Disparate Impact", classified_metric.disparate_impact())
    print("---------------------------------------------")

for protected_attribute in protected_attributes:

    # prepare the dataset in the format that aif360 requires
    dataset = createBinaryDataset(df, df['Y_TRUE'].to_frame(), protected_attribute)
    df_pred = df['Y_PRED'].to_frame()
    df_pred.rename(columns={'Y_PRED': 'Y_TRUE'}, inplace=True)
    dataset_pred = createBinaryDataset(df, df_pred, protected_attribute)

    # creates an object that computes metrics for binary classification
    index = dataset_pred.protected_attribute_names.index(protected_attribute)
    privileged_groups = [{protected_attribute: dataset_pred.privileged_protected_attributes[index][0]}]
    unprivileged_groups = [{protected_attribute: dataset_pred.unprivileged_protected_attributes[index][0]}]

    print("=============================================")
    print(protected_attribute)
    print("=============================================")
    print("Before bias mitigation: ")

    # print the metrics
    runMetrics(dataset, dataset_pred, unprivileged_groups, privileged_groups)

    # postprocessing bias mitigation using calibrated equal odds and transforming the predictions
    cpp = CalibratedEqOddsPostprocessing(privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)
    cpp = cpp.fit(dataset, dataset_pred)
    dataset_pred_transformed = cpp.predict(dataset_pred)

    print("After bias mitigation: ")

    # print the metrics
    runMetrics(dataset, dataset_pred_transformed, unprivileged_groups, privileged_groups)
    
    print()

GENDER
Before bias mitigation: 
Accuracy:  0.8995673671199012
Error Rate Difference:  0.015614395546898363
False Discovery Rate Difference:  0.032453245324532476
False Omission Rate Difference:  0.012607172328908914
Differential Fairness Bias Amplification:  -0.007940972350255926
Disparate Impact 1.112777712795602
---------------------------------------------
After bias mitigation: 
Accuracy:  0.8989493201483313
Error Rate Difference:  0.014501429492362994
False Discovery Rate Difference:  0.04406130268199232
False Omission Rate Difference:  0.008566015978914432
Differential Fairness Bias Amplification:  0.1404790327680172
Disparate Impact 1.2918453907167333
---------------------------------------------

LANGUAGE
Before bias mitigation: 
Accuracy:  0.8995673671199012
Error Rate Difference:  0.008343244151104434
False Discovery Rate Difference:  -0.06464285714285717
False Omission Rate Difference:  0.012209092270952848
Differential Fairness Bias Amplification:  -0.07482286044713238
Disp